
    ## 导学单元

    ### 课程目标
    - 认识真实流域的能量-水量平衡问题，并理解Budyko曲线在水文气候分区中的作用。
    - 熟悉Notebook与脚本 `examples/01_real_data_workflow.py` 的配合方式，便于在命令行与交互环境间切换。

    ### 前置知识
    - 基础 Python（NumPy/Pandas）与 Matplotlib/Seaborn 绘图经验。
    - 对能量-水量平衡、蒸散发与降水等基本水文气候概念的认知。

    ### 所需包和运行环境
    - Python 3.9+，建议在虚拟环境或 Conda 环境中运行 Jupyter Notebook。
    - 依赖包主要列于项目 `requirements.txt`，常用库包括 NumPy、Pandas、Matplotlib、Seaborn、Xarray、SciPy（按需）。
    - 如需加速或批量运行，可直接在命令行调用 `examples/01_real_data_workflow.py` 以生成对应输出。

    ### 学习路径与章节骨架
    1. 背景/概念：认识真实流域的能量-水量平衡问题，并理解Budyko曲线在水文气候分区中的作用。
    2. 数据加载与预览：理解数据来源、字段含义与基本统计特征。
    3. 核心方法步骤（配合 `examples/01_real_data_workflow.py`）：调用脚本中的数据读取、预处理、Budyko指标计算与曲线拟合函数，串联完整分析流程。
    4. 结果可视化：通过散点、拟合曲线与残差可视化，检视模型适配度与异常点。
    5. 反思与扩展练习：思考不同气候区或时间分辨率下的曲线差异，尝试替换其他真实数据集。

    ### 你将学到什么
    - 如何基于真实观测数据构建Budyko分析数据集并完成必要清洗。
- 利用脚本函数快速完成潜在蒸散发、干燥度指数等关键指标计算。
- 通过可视化诊断拟合效果并识别偏离Budyko假设的情景。

    ### 本节完成标志
    - [ ] 成功运行脚本并生成Budyko散点与拟合曲线图。
- [ ] 理解每个中间数据表/变量的含义，能解释曲线偏离的可能原因。


# 01_real_data_workflow.py Notebook

完整的真实世界Budyko分析工作流程

**目标**：
展示如何使用本代码库处理真实的、多源的科学数据，
完成从数据加载到综合分析的全流程Budyko分析。

**数据来源**（模拟）：
1. 站点径流数据（CSV） - Q，水量平衡的锚点
2. 格点气象数据（NetCDF） - P, T, RH, 风速, 辐射
3. 遥感LAI数据（NetCDF） - MODIS LAI
4. 遥感TWS数据（NetCDF） - GRACE TWS
5. CO2浓度数据 - Mauna Loa观测
6. 人为驱动因子（CSV） - 灌溉、土地利用等

**分析范式**：
1. Ibrahim (2025) 偏差分析 - 时间稳定性
2. Jaramillo (2022) 轨迹分析 - 空间运动方向
3. He (2023) 3D框架 - ΔS的作用

**核心创新**：
对比标准PET（Penman-Monteith）与创新PET（考虑LAI和CO2）的差异

In [1]:
# examples/01_real_data_workflow.py

import sys
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from pathlib import Path

# 在 Notebook 中，使用 Path.cwd() 获取当前工作目录
# 假设 notebook 在 examples/ 目录下，src/ 在父目录
project_root = Path.cwd()
if project_root.name == 'examples':
    # 如果当前在 examples 目录，向上一级找到项目根目录
    project_root = project_root.parent
sys.path.insert(0, str(project_root / 'src'))

# 导入中文字体配置
try:
    from utils.plotting_config import setup_chinese_fonts
    setup_chinese_fonts()
except ImportError:
    # 备用配置
    plt.rcParams['font.sans-serif'] = ['Microsoft YaHei', 'SimHei', 'DejaVu Sans']
    plt.rcParams['axes.unicode_minus'] = False

print(f"✓ 环境准备完成")
print(f"项目根目录: {project_root}")

✓ 中文字体配置已应用
✓ 中文字体配置已应用
✓ 环境准备完成
项目根目录: c:\Users\ASUS\OneDrive\SCI\Github\Budyko-Analysis


In [2]:
# ========================
# 导入本代码库的模块
# ========================
from data_processing.basin_processor import BasinDataProcessor
from data_processing.grace_lai_processor import GRACEDataLoader, LAIDataLoader, CO2DataLoader

from models.pet_models import PenmanMonteithPET
from models.pet_lai_co2 import PETWithLAICO2, PETComparator

from budyko.water_balance import WaterBalanceCalculator
from budyko.curves import BudykoCurves
from budyko.deviation import DeviationAnalysis, TemporalStability, MarginalDistribution
from budyko.trajectory_jaramillo import TrajectoryAnalyzer, MovementStatistics

from analysis.deviation_attribution import DeviationAttributor

from visualization.budyko_plots import BudykoVisualizer, create_comprehensive_figure

print("✓ Budyko 分析模块导入成功！")

✓ Budyko 分析模块导入成功！


# 第一部分：数据加载与预处理

在实际项目中，这些数据通常来自：
- **径流数据（Q）**：水文站观测
- **气象数据（P, T, RH, 风速, 辐射）**：气象站或再分析数据（如 CMFD）
- **遥感数据（LAI, TWS）**：卫星产品（MODIS, GRACE）
- **CO₂ 数据**：全球观测站（如 Mauna Loa）

为了教学目的，我们创建一个辅助函数来生成模拟数据。

In [3]:
print("="*80)
print("【第一部分】数据加载与预处理")
print("-"*80)

# 生成模拟数据
print("生成模拟流域数据...")
years = np.arange(1980, 2021)  # 1980-2020，共41年
n_years = len(years)
basin_id = 'Basin_001'

np.random.seed(42)

# 降水P：500-1500mm，有年际变率
P_mean = 800
P = P_mean + 200 * np.sin(2 * np.pi * years / 30) + np.random.normal(0, 100, n_years)
P = np.maximum(P, 300)

# 径流Q：基于P，径流系数0.3-0.5
RC_base = 0.4
Q = P * (RC_base + 0.1 * np.sin(2 * np.pi * years / 20) + np.random.normal(0, 0.05, n_years))
Q = np.clip(Q, 50, P * 0.9)

# 温度T：10-20°C，有升温趋势
T = 15 + 0.03 * (years - 1980) + 3 * np.sin(2 * np.pi * (np.arange(n_years) % 12) / 12) + np.random.normal(0, 1, n_years)

# 相对湿度RH：60-80%
RH = 70 + np.random.normal(0, 5, n_years)
RH = np.clip(RH, 40, 95)

# 风速u2：1-3 m/s
u2 = 2 + np.random.normal(0, 0.5, n_years)
u2 = np.maximum(u2, 0.5)

# 辐射Rn：100-200 W/m²
Rn = 150 + 30 * np.sin(2 * np.pi * (np.arange(n_years) % 12) / 12) + np.random.normal(0, 10, n_years)
Rn = np.maximum(Rn, 50)

# LAI：1-5 m²/m²，有增长趋势（绿化）
LAI = 2.5 + 0.5 * (years - 1980) / 40 + 0.5 * np.sin(2 * np.pi * (np.arange(n_years) % 12) / 12) + np.random.normal(0, 0.3, n_years)
LAI = np.clip(LAI, 0.5, 6)

# CO2：从340ppm（1980）增长到415ppm（2020）
CO2 = 340 + 1.9 * (years - 1980) + np.random.normal(0, 2, n_years)

# TWS：陆地水储量变化（GRACE, 2002-2020）
TWS = np.zeros(n_years)
TWS[years >= 2002] = 50 * np.sin(2 * np.pi * (years[years >= 2002] - 2002) / 10) + np.random.normal(0, 20, sum(years >= 2002))
TWS[years < 2002] = np.nan

# ΔS：储量变化
delta_S = np.diff(TWS, prepend=np.nan)

# 人为驱动因子：灌溉面积比例
irrigation_ratio = 0.1 + 0.2 * (years - 1980) / 40 + np.random.normal(0, 0.02, n_years)
irrigation_ratio = np.clip(irrigation_ratio, 0, 0.5)

# 构建DataFrame
data = pd.DataFrame({
    'year': years,
    'basin_id': basin_id,
    'P': P,
    'Q': Q,
    'T': T,
    'RH': RH,
    'u2': u2,
    'Rn': Rn,
    'LAI': LAI,
    'CO2': CO2,
    'TWS': TWS,
    'delta_S': delta_S,
    'irrigation_ratio': irrigation_ratio
})

print(f"✓ 模拟数据生成完成：{n_years}年，流域ID={basin_id}")
print(f"\n数据预览：")
data.head()

【第一部分】数据加载与预处理
--------------------------------------------------------------------------------
生成模拟流域数据...
✓ 模拟数据生成完成：41年，流域ID=Basin_001

数据预览：


,year,basin_id,P,Q,T,RH,u2,Rn,LAI,CO2,TWS,delta_S,irrigation_ratio
0,1980,Basin_001,849.671415,347.148903,16.477894,62.990745,2.481688,140.621750,2.357516,341.176634,NaN,NaN,0.071851
1,1981,Basin_001,827.755908,351.895000,16.011730,72.934285,2.206390,170.150353,2.566501,342.461984,NaN,NaN,0.090631
2,1982,Basin_001,946.116182,419.813833,16.849583,80.952278,2.411030,181.118622,3.487649,342.554601,NaN,NaN,0.105731
3,1983,Basin_001,1069.860036,435.406930,17.588243,65.047318,2.948396,185.150477,3.158995,345.283755,NaN,NaN,0.121218
4,1984,Basin_001,925.213628,424.778012,18.633478,67.168511,1.877306,214.508077,2.604748,346.613998,NaN,NaN,0.149507
